In [2]:
from Bio import SeqIO
import re

out_src = 'datasets/uniprot/' #specify output folder

pattern = re.compile("[^ARNDCEQGHILKMFPSTWYV]") #no B asaparctic acid or Z glutamic acid (or "X" or "O")
standard = []
non_standard= []
for idx, record in enumerate(SeqIO.parse("datasets/uniprot/uniprot_len_50_std_only_non_antimicrobial.fasta", "fasta")):
    if pattern.search(str(record.seq)):
        non_standard.append(idx) #saving the index of the non-standard AAs for future reference
    else:
        standard.append(record.seq)
with open(out_src+'non_standard_index.txt', 'w') as f:
    for index in non_standard: f.write(str(index)+'\n')
f.close()
print("Number of sequences with only standard AA's: ",len(standard), "\t Number with non-standard AA's",len(non_standard))

Number of sequences with only standard AA's:  3658013 	 Number with non-standard AA's 27439


In [3]:
len(standard) != len(set(standard)) #sets do not allow duplicate values so we check if both lists are the same size

True

In [9]:
#Throw away the duplicates by keeping only the set
standard = set(standard)

In [3]:
out_src = 'datasets/uniprot_v2/' #specify output folder
smile_list=[]
with open('datasets\\uniprot_v2\\sunistar.txt', 'r') as f:
    for line in f:
        smile_list.append(line.strip('\n'))
smile_list

['LE',
 'EL',
 'RW',
 'YL',
 'FF',
 'GP',
 'KR',
 'WW',
 'CS',
 'DD',
 'DE',
 'DQ',
 'EW',
 'FA',
 'FG',
 'FK',
 'FP',
 'FR',
 'FV',
 'FW',
 'FY',
 'GF',
 'GG',
 'GV',
 'HG',
 'KA',
 'KF',
 'KG',
 'KL',
 'KP',
 'KW',
 'KY',
 'LL',
 'LP',
 'LV',
 'LW',
 'LY',
 'PF',
 'PG',
 'PK',
 'PL',
 'PP',
 'PY',
 'RP',
 'RR',
 'VF',
 'VG',
 'VV',
 'WE',
 'WL',
 'WP',
 'WY',
 'YA',
 'YF',
 'YR',
 'AA',
 'AD',
 'AF',
 'AG',
 'AH',
 'AI',
 'AL',
 'AP',
 'AR',
 'AV',
 'AW',
 'AY',
 'CF',
 'DA',
 'DF',
 'DG',
 'DK',
 'DL',
 'DM',
 'DP',
 'DV',
 'DW',
 'DY',
 'EA',
 'EE',
 'EG',
 'EI',
 'EK',
 'EP',
 'ER',
 'EV',
 'EY',
 'FE',
 'FI',
 'FL',
 'FQ',
 'GA',
 'GD',
 'GE',
 'GH',
 'GI',
 'GK',
 'GL',
 'GM',
 'GQ',
 'GR',
 'GS',
 'GT',
 'GW',
 'GY',
 'HK',
 'HL',
 'HP',
 'HY',
 'IA',
 'ID',
 'IE',
 'IF',
 'IG',
 'II',
 'IK',
 'IL',
 'IN',
 'IP',
 'IQ',
 'IR',
 'IS',
 'IT',
 'IV',
 'IW',
 'IY',
 'KD',
 'KE',
 'KH',
 'KI',
 'KK',
 'KS',
 'KT',
 'KV',
 'LA',
 'LD',
 'LF',
 'LG',
 'LI',
 'LK',
 'LN',
 'LQ',
 'MA',

In [4]:
import random
random.seed(0) # set the seed so shuffling is reproducible
#saving the standard AA only list to a new file.
# shuffled_standard = random.sample(standard, len(standard))
shuffled_standard = random.sample(smile_list, len(smile_list))

# with open(out_src+'peptide_train+test_no_shuff.txt', 'w') as f:
#     for idx, line in enumerate(standard):
#         f.write(str(line)+'\n')
# f.close()

with open(out_src+'peptide_train.txt', 'w') as f:
    f.write('peptides'+'\n')
    for idx,line in enumerate(shuffled_standard):
        if idx <round(0.05*len(shuffled_standard)):
            continue # the first 5% will be used for testing
        f.write(str(line)+'\n')
f.close()

with open(out_src+'peptide_test.txt', 'w') as f:
    f.write('peptides'+'\n')
    for idx,line in enumerate(shuffled_standard):
        if idx <round(0.05*len(shuffled_standard)):
            f.write(str(line)+'\n')
        else:
            break
f.close()

In [5]:
import sys
sys.path.append('../../') #allows import from 2 dirs above the CWD

inputs = 'datasets/uniprot_v2/peptide_train.txt'
max_len = 50 #max length of the sequence
save_dir = out_src
vocab_name = 'char_dict_peptide'
weights_name = 'char_weights_peptide'

%run ../../scripts/build_vocab.py --inputs $inputs --max_len $max_len --save_dir $save_dir --vocab_name $vocab_name --weights_name $weights_name

building dictionary...
calculating weights...
{'I': 510494, 'P': 367825, 'R': 533012, 'Q': 290022, 'G': 532833, 'L': 812294, 'E': 429551, 'K': 537978, 'A': 605445, 'F': 373851, 'S': 575750, 'N': 331790, 'C': 200202, 'H': 192967, 'T': 430551, 'V': 524701, 'W': 128388, 'M': 351388, 'Y': 241496, 'D': 339720, '_': 6138942, '<end>': 288984}
